In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [17]:
diag = pd.read_csv('CSV files\\diagnoses_table.csv')

In [18]:
diag.head(10)

,subject_id,hadm_id,icd_code,icd_version
0,13757356,22748649,V1083,9
1,13781225,21231709,3572,9
2,13786252,21097730,36250,9
3,13799390,22090873,V8535,9
4,13870030,28791864,2572,9
5,13872674,25090386,60784,9
6,13886433,29507168,7840,9
7,13916179,28451236,2858,9
8,13962855,24639924,2762,9
9,13993861,28782751,2859,9


In [19]:
adm = pd.read_csv('CSV files\\admissions_table.csv')

In [20]:
adm.head(10)

,subject_id,hadm_id,admittime,dischtime
0,10106244,26713233,2147-05-09 10:34:00,2147-05-12 13:43:00
1,10106244,26738324,2147-05-09 10:34:00,2147-05-12 13:43:00
2,10106244,22486493,2147-05-09 10:34:00,2147-05-12 13:43:00
3,15443666,22294409,2168-12-30 23:30:00,2169-01-05 16:02:00
4,15443666,24613632,2168-12-30 23:30:00,2169-01-05 16:02:00
5,15443666,25959308,2168-12-30 23:30:00,2169-01-05 16:02:00
6,15443666,22121441,2168-12-30 23:30:00,2169-01-05 16:02:00
7,15443666,27961368,2168-12-30 23:30:00,2169-01-05 16:02:00
8,15443666,29035604,2168-12-30 23:30:00,2169-01-05 16:02:00
9,16299919,29997664,2193-05-15 08:37:00,2193-05-17 16:03:00


In [21]:
diag_df = diag.merge(adm, on=['subject_id', 'hadm_id'])

In [22]:
diag_df.head(10)

,subject_id,hadm_id,icd_code,icd_version,admittime,dischtime
0,13757356,22748649,V1083,9,2142-11-11 21:51:00,2142-11-12 12:34:00
1,13757356,22748649,V1083,9,2148-04-01 21:22:00,2148-04-03 14:32:00
2,13757356,22748649,V1083,9,2148-04-03 19:53:00,2148-04-05 14:08:00
3,13757356,22748649,V1083,9,2149-09-18 22:18:00,2149-09-19 17:08:00
4,13757356,22748649,V1083,9,2148-07-21 06:40:00,2148-07-30 17:00:00
5,13757356,22748649,V1083,9,2147-06-19 19:56:00,2147-06-21 17:46:00
6,13757356,22748649,V1083,9,2148-02-25 20:41:00,2148-03-01 16:00:00
7,13757356,22748649,V1083,9,2148-06-29 01:10:00,2148-07-03 15:50:00
8,13781225,21231709,3572,9,2130-02-27 04:47:00,2130-03-05 16:35:00
9,13786252,21097730,36250,9,2142-12-01 01:10:00,2142-12-11 15:11:00


In [23]:
diag_df.shape

(44474993, 6)

In [24]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Load your combined CSV or DataFrame
df = diag_df.copy()

st.title("ICD Code Timeline by Subject")

# Optional: filter by subject_id or hadm_id
subject_ids = df['subject_id'].unique()
selected_subject = st.selectbox("Select Subject ID", subject_ids)

filtered_df = df[df['subject_id'] == selected_subject]

# Create timeline plot
fig = px.timeline(
    filtered_df,
    x_start="admittime",
    x_end="dischtime",
    y="icd_code",
    color="icd_version",
    title=f"ICD Diagnosis Timeline for Subject {selected_subject}",
    hover_data=["hadm_id"]
)

fig.update_yaxes(autorange="reversed")  # Timeline style
st.plotly_chart(fig, use_container_width=True)


2025-05-07 15:35:08.923 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 15:35:08.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 15:35:09.132 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 15:35:09.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 15:35:09.157 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 15:35:09.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 15:35:09.165 Session state does not function when running a script without `streamlit run`
2025-05-07 15:35:09.169 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-07 15:35

DeltaGenerator()